# Stateful models

A “stateful model” is a model that implicitly preserves data between two consecutive inference calls. The tensors saved from one run are kept in an internal memory buffer called a “state” or a “variable” and may be passed to the next run, while never being exposed as model output. In contrast, for a “stateless” model to pass data between runs, all produced data is returned as output and needs to be handled by the application itself for reuse at the next execution.

![avatar](stateless%20vs%20stateful.png)

## Export a stateless INT8 model

In [ ]:
!python convert.py --model_id THUDM/chatglm3-6b --disable-stateful -c INT8 --output_dir chatglm3-6b-ov-int8-stateless -p FP16

## Export a stateless INT4 model

In [ ]:
!python convert.py --model_id THUDM/chatglm3-6b --disable-stateful -c INT4_SYM --ratio 0.8 --group_size 128 --output_dir chatglm3-6b-ov-int4-stateless -p FP16

## Export a stateful INT4 model

In [ ]:
!python convert.py --model_id THUDM/chatglm3-6b --stateful -c INT4_SYM --ratio 0.8 --group_size 128 --output_dir chatglm3-6b-ov-int4-stateful -p FP16

Paramters:
* `--model_id` - model_id for downloading from huggngface_hub (https://huggingface.co/models) or path with directory where pytorch model located.
* `--output_dir` - output directory for saving OpenVINO model
* `--precision` - (optional, default FP32), precision for model conversion FP32 or FP16
* `--save_orig` - flag for saving original pytorch model, model will be located in `<output_dir>/pytorch` subdirectory.
* `--compress_weights` - The weight compression option, INT8 - INT8 weights, 4BIT_DEFAULT - for 4-bit weights compression with predefined configuration, INT4_SYM - for INT4 compressed weights with symmetric quantization, INT4_ASYM - for INT4 compressed weights with assymetric quantization. You can specify multiple backends separated by a space.
* `--compress_weights_backends` - (optional, default openvino) backends for weights compression, this option has an effect only with `--compress_weights`. You can specify multiple backends separated by a space.
* `--ratio` - Compression ratio between primary and backup precision, e.g. INT4/INT8.
* `--group_size` - Size of the group of weights that share the same quantization parameters


If you have difficulty accessing `huggingface`, you can try to use `mirror-hf` to download:

  * Linux

    ```
    export HF_ENDPOINT=https://hf-mirror.com
    ```
   
  * Windows Powershell

    ```
    $env:HF_ENDPOINT = "https://hf-mirror.com"
    ```
  
  * Download model
  
    ```
    huggingface-cli download --resume-download --local-dir-use-symlinks False THUDM/chatglm3-6b --local-dir {your_path}/chatglm3-6b
    ```

# Why stateful models ?

Text generation is a good usage example of stateful models, as it requires multiple inference calls to output a complete sentence, each run producing a single output token. Information from one run is passed to the next inference as a context, which may be handled by a stateful model natively. Potential benefits for this, as well as other scenarios, may be:

model execution speedup - data in states is stored in the optimized form for OpenVINO plugins, which helps to execute the model more efficiently. Importantly, requesting data from the state too often may reduce the expected performance gains or even lead to losses. Use the state mechanism only if the state data is not accessed very frequently.

user code simplification - states can replace code-based solutions for such scenarios as giving initializing values for the first inference call or copying data from model outputs to inputs. With states, OpenVINO will manage these cases internally, additionally removing the potential for additional overhead due to data representation conversion.

data processing - some use cases require processing of data sequences. When such a sequence is of known length and short enough, you can process it with RNN-like models that contain a cycle inside. When the length is not known, as in the case of online speech recognition or time series forecasting, you can divide the data in small portions and process it step-by-step, which requires addressing the dependency between data portions. States fulfil this purpose well: models save some data between inference runs, when one dependent sequence is over, the state may be reset to the initial value and a new sequence can be started.

## Run benchmark for stateless INT8 model

In [ ]:
!python benchmark.py -m "chatglm3-6b-ov-int4-stateful\pytorch\dldt\compressed_weights\OV_FP16-INT8" -n 5 -d gpu -p "若我有一亿美元，在人工智能盛行的今天，我怎样投资才能收益最大化？"  -ic 64

## Run benchmark for stateless INT4 model

In [ ]:
!python benchmark.py -m "chatglm3-6b-ov-int4-stateless\pytorch\dldt\compressed_weights\OV_FP16-INT4_SYM" -n 5 -d gpu -p "若我有一亿美元，在人工智能盛行的今天，我怎样投资才能收益最大化？"  -ic 64

## Run benchmark with stateful INT4  model

In [ ]:
!python benchmark.py -m "chatglm3-6b-ov-int4-stateful\pytorch\dldt\compressed_weights\OV_FP16-INT4_SYM" -n 5 -d gpu -p "若我有一亿美元，在人工智能盛行的今天，我怎样投资才能收益最大化？"  -ic 64

Paramters:
* `-m` - model path
* `-d` - inference device (default=cpu)
* `-r` - report csv
* `-f` - framework (default=ov)
* `-p` - interactive prompt text
* `-pf` - path of JSONL file including interactive prompts
* `-n` - number of benchmarking iterations, if the value greater 0, will exclude the first iteration. (default=0)